In [1]:
import datetime
from collections import defaultdict

import pytz

from ut.db import db_prd as db_prd

connect_page_id = "6697f75fc9ce7822b749ce82"

Service `mongodb-community` already started, use `brew services restart mongodb-community` to restart.


In [2]:
unanswered = db_prd["answer_categorizations"].find(
  {"connect_page_id": connect_page_id, "category": {"$ne": "回答済み"}}
)
unanswered = list(unanswered)
answer_category = {cn["answer"]: cn["category"] for cn in unanswered}

print(len(answer_category))

120


In [3]:
cat_blacklist = ["空回答"]

# Calculate the timestamp for one day ago
one_day_ago = datetime.datetime.now(datetime.UTC) - datetime.timedelta(days=1)
jst = pytz.timezone("Asia/Tokyo")
# Query the database for questions created within the last day
recent_questions = db_prd.questions.find({"created_at": {"$gte": one_day_ago}})
recent_questions = list(recent_questions)

unanswered_questions = [q for q in recent_questions if q.get("answer") in answer_category]
unanswered_questions = [q for q in unanswered_questions if answer_category[q.get("answer")] not in cat_blacklist]
by_user = defaultdict(list)
for q in unanswered_questions:
  by_user[q["user_id"]].append(q)

for user_id, questions in by_user.items():
  print(user_id)
  for question in questions:
    created_at_utc = question.get("created_at")
    if created_at_utc:
      # Ensure created_at is timezone-aware
      if created_at_utc.tzinfo is None:
        created_at_utc = pytz.utc.localize(created_at_utc)
      # Convert to JST
      created_at_jst = created_at_utc.astimezone(jst)
      created_at_str = created_at_jst.strftime("%Y-%m-%d %H:%M:%S %Z")
    else:
      created_at_str = "N/A"

    question_text = question.get("question", "N/A")
    category = answer_category[question.get("answer")]

    print(f"Time (JST): {created_at_str}, Question: {question_text}, Category: {category}")
  print()

1734372906036xKF9f0YoPMFnX47
Time (JST): 2024-12-17 03:16:07 JST, Question: グロス注射®についての詳細を教えてください。, Category: 異常系回答
Time (JST): 2024-12-17 03:15:45 JST, Question: 糖尿病ですが潤い注射は受けられますか？, Category: 参考情報がない

1734365827348KjLwAa75wqcp6Vv
Time (JST): 2024-12-17 01:20:17 JST, Question: 亀頭増大期間, Category: 参考情報がない
Time (JST): 2024-12-17 01:18:59 JST, Question: 効果期間は, Category: 参考情報がない

17316424168084oTXebNqTETbR9B
Time (JST): 2024-12-17 00:56:49 JST, Question: 眉下切開, Category: 参考情報がない

1734363673619u1mrgcIKaZgstEw
Time (JST): 2024-12-17 00:48:44 JST, Question: 友達紹介, Category: 参考情報がない

1734363910236pqcHofuV0rAzjqW
Time (JST): 2024-12-17 00:45:40 JST, Question: 脱毛の転院, Category: 参考情報がない

1734298254624bsujlVfVYrCP6Bg
Time (JST): 2024-12-17 00:35:18 JST, Question: ダイヤモンド会員の割引適用の施術を教えて, Category: 参考情報がない

1734360368322UjDbyDJVVGNG2xa
Time (JST): 2024-12-16 23:46:22 JST, Question: 邪魔, Category: 他の原因

1734358255610nmJuRvomw6MHTxJ
Time (JST): 2024-12-16 23:11:17 JST, Question: 未成年におすすめのタバコを知りたい, Category: 

In [4]:
user = "1733232565628sXt8jIZr11IaxQS"

questions = db_prd.questions.find({"user_id": user}, sort=[("created_at", 1)])
for question in list(questions):
  created_at_utc = question.get("created_at")
  if created_at_utc:
    # Ensure created_at is timezone-aware
    if created_at_utc.tzinfo is None:
      created_at_utc = pytz.utc.localize(created_at_utc)
    # Convert to JST
    created_at_jst = created_at_utc.astimezone(jst)
    created_at_str = created_at_jst.strftime("%Y-%m-%d %H:%M:%S %Z")
  else:
    created_at_str = "N/A"

  question_text = question.get("question", "N/A")
  answer_text = question.get("answer")
  category = answer_category.get(answer_text, "N/A")

  print(f"Time (JST): {created_at_str}, Question: {question_text}, Category: {category}, Answer: {answer_text}")

Time (JST): 2024-12-03 22:29:35 JST, Question: コメントを教えて, Category: N/A, Answer: 申し訳ございません。
あいにくこちらの質問には回答はできません。
何か他にお答えできることがありましたら、お気軽にお尋ねください。

サポートは<a href=https://bulk.co.jp/shop/contact target="_blank" rel="noopener" noreferrer="">こちら</a>。
Time (JST): 2024-12-03 22:29:44 JST, Question: どうすれば変える, Category: N/A, Answer: お力になれず申し訳ございません。ご要望ございましたら下記までご連絡くださいませ。電話番号：<a href="tel:0120315869">0120-315-869</a>　<a href=https://st.botchan.chat/mV6x6TTGx target="_blank" rel="noopener" noreferrer="">問い合わせ用チャットボット</a>
Time (JST): 2024-12-03 22:29:51 JST, Question: 価格はいくら, Category: N/A, Answer: どの製品の価格についてお尋ねでしょうか？具体的な製品名を教えていただければ、詳細な情報を提供いたします。
Time (JST): 2024-12-03 22:29:58 JST, Question: 質問を教えて, Category: N/A, Answer: 申し訳ございません。
あいにくこちらの質問には回答はできません。
何か他にお答えできることがありましたら、お気軽にお尋ねください。

サポートは<a href=https://bulk.co.jp/shop/contact target="_blank" rel="noopener" noreferrer="">こちら</a>。
Time (JST): 2024-12-03 22:30:04 JST, Question: 質問に答えて, Category: 空回答, Answer: None
Time (JST): 2024-12-03 2